
# 문제 4-1 : 카페 메뉴 도구

## 0) 환경 점검

In [1]:

import os, sys, platform
print("Python:", sys.version)
print("Platform:", platform.platform())
print("OPENAI_API_KEY set:", bool(os.getenv("OPENAI_API_KEY")))
print("TAVILY_API_KEY set:", bool(os.getenv("TAVILY_API_KEY")))


Python: 3.12.7 | packaged by Anaconda, Inc. | (main, Oct  4 2024, 13:17:27) [MSC v.1929 64 bit (AMD64)]
Platform: Windows-11-10.0.26100-SP0
OPENAI_API_KEY set: True
TAVILY_API_KEY set: True


## 1) 데이터 로드

In [15]:

from pathlib import Path

DATA_DIR = Path.cwd() / "data"
DATA_DIR.mkdir(parents=True, exist_ok=True)
menu_path = DATA_DIR / "cafe_menu_data.txt"

assert menu_path.exists(), f"파일이 없습니다: {menu_path}"
txt = menu_path.read_text(encoding="utf-8")
print("파일 경로:", menu_path)
print("\n=== 미리보기 ===\n", txt[:500], "..." if len(txt)>500 else "")


파일 경로: c:\mylangchain\mylangchain-app\src\mylangchain_app\0example\data\cafe_menu_data.txt

=== 미리보기 ===
 1. 아메리카노
   • 가격: ₩4,500
   • 주요 원료: 에스프레소, 뜨거운 물
   • 설명: 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피입니다. 원두 본연의 맛을 가장 잘 느낄 수 있으며, 깔끔하고 깊은 풍미가 특징입니다. 설탕이나 시럽 추가 가능합니다.

2. 카페라떼
   • 가격: ₩5,500
   • 주요 원료: 에스프레소, 스팀 밀크
   • 설명: 진한 에스프레소에 부드럽게 스팀한 우유를 넣어 만든 대표적인 밀크 커피입니다. 크리미한 질감과 부드러운 맛이 특징이며, 다양한 시럽과 토핑 추가가 가능합니다. 라떼 아트로 시각적 즐거움도 제공합니다.

3. 카푸치노
   • 가격: ₩5,000
   • 주요 원료: 에스프레소, 스팀 밀크, 우유 거품
   • 설명: 에스프레소, 스팀 밀크, 우유 거품이 1:1:1 비율로 구성된 이탈리아 전통 커피입니다. 진한 커피 맛과 부드러운 우유 거품의 조화가 일품이며, 계피 파우더를 뿌려 제공합니다.

4. 바 ...


## 2) 벡터 DB 구축 (FAISS, OpenAI Embeddings)

In [16]:

import re, json
from pathlib import Path
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.schema import Document

DB_DIR = Path.cwd() / "db" / "cafe_db"
DB_DIR.mkdir(parents=True, exist_ok=True)

# --- 문서 분할 ---
# 기본 포맷 가정: 메뉴 블록 간 빈 줄로 구분, 각 블록 첫 줄=메뉴명
blocks = [b.strip() for b in re.split(r"\n\s*\n", txt) if b.strip()]

docs = []
for block in blocks:
    lines = block.splitlines()
    menu_name = lines[0].strip() if lines else "Unknown"
    docs.append(Document(page_content=block, metadata={"menu_name": menu_name}))

print(f"총 문서 블록 수: {len(docs)}")

# --- 임베딩 & 인덱스 ---
emb = OpenAIEmbeddings(model="text-embedding-3-small")
vs = FAISS.from_documents(docs, emb)
vs.save_local(str(DB_DIR))
print("✅ FAISS index saved to:", DB_DIR)


c:\Users\user\AppData\Local\pypoetry\Cache\virtualenvs\mylangchain-app-SBe-Yh6W-py3.12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


총 문서 블록 수: 10
✅ FAISS index saved to: c:\mylangchain\mylangchain-app\src\mylangchain_app\0example\db\cafe_db


## 3) 도구 정의

In [17]:

import json, wikipedia
from langchain.tools import tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from pathlib import Path

DB_DIR = Path.cwd() / "db" / "cafe_db"
emb = OpenAIEmbeddings(model="text-embedding-3-small")

@tool("tavily_search_func", return_direct=False)
def tavily_search_func(query: str) -> str:
    """웹에서 최신 정보를 검색합니다 (Tavily). 입력: 검색어(str). 출력: 요약 문자열."""
    tavily = TavilySearchResults(max_results=5)
    results = tavily.invoke({"query": query})
    out = []
    for i, r in enumerate(results, 1):
        out.append(f"[{i}] {r.get('url','')}\n{r.get('content','')[:300]}...")
    return "\n\n".join(out) if out else "검색 결과가 없습니다."

@tool("wiki_summary", return_direct=False)
def wiki_summary(topic: str) -> str:
    """위키피디아에서 주제 요약을 제공합니다. 입력: 주제(str). 출력: 요약 문자열."""
    try:
        wikipedia.set_lang("ko")
        return wikipedia.summary(topic, sentences=3, auto_suggest=False, redirect=True)
    except Exception as e:
        return f"위키 요약 실패: {e}"

@tool("db_search_cafe_func", return_direct=False)
def db_search_cafe_func(query: str) -> str:
    """로컬 카페 메뉴 DB에서 유사한 항목을 검색합니다. 입력: 쿼리(str). 출력: JSON 문자열(List[Document])."""
    local_vs = FAISS.load_local(str(DB_DIR), embeddings=emb, allow_dangerous_deserialization=True)
    found = local_vs.similarity_search(query, k=4)
    payload = [{"page_content": d.page_content, "metadata": d.metadata} for d in found]
    return json.dumps(payload, ensure_ascii=False)


## 4) LLM 바인딩 및 @chain 워크플로우

In [18]:

from typing import Dict, Any
from langchain_openai import ChatOpenAI
from langchain_core.runnables import chain
from langchain_core.messages import AIMessage

TOOLS = [tavily_search_func, wiki_summary, db_search_cafe_func]
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
llm_with_tools = llm.bind_tools(TOOLS)

@chain
def cafe_tool_chain(question: str) -> Dict[str, Any]:
    ai: AIMessage = llm_with_tools.invoke(question)
    calls = getattr(ai, "tool_calls", []) or []
    tool_results = []

    if calls:
        for c in calls:
            name = c["name"]
            args = c.get("args", {})
            if name == "tavily_search_func":
                out = tavily_search_func.invoke(args.get("query", question))
            elif name == "wiki_summary":
                out = wiki_summary.invoke(args.get("topic", question))
            elif name == "db_search_cafe_func":
                out = db_search_cafe_func.invoke(args.get("query", question))
            else:
                out = f"알 수 없는 도구: {name}"
            tool_results.append({"tool": name, "output": out})

        ctx = "\n\n".join([f"[{r['tool']}]\n{r['output']}" for r in tool_results])
        final = llm.invoke(f"""사용자 질문: {question}
아래 도구 결과를 참고하여 한국어로 간결하고 정확하게 답하세요.

도구 결과:
{ctx}
""")
        return {"answer": final.content, "tool_calls": calls, "tool_results": tool_results}
    else:
        return {"answer": ai.content, "tool_calls": [], "tool_results": []}


## 5) 테스트 — “아메리카노의 가격과 특징은 무엇인가요?”

In [19]:

out = cafe_tool_chain.invoke("아메리카노의 가격과 특징은 무엇인가요?")
print("=== 최종 답변 ===\n", out["answer"])
print("\n=== 도구 호출 내역 ===\n", out["tool_calls"])
print("\n=== 도구 실행 결과(요약) ===")
for r in out["tool_results"]:
    print(f"- {r['tool']}: {str(r['output'])[:180]}...")


c:\Users\user\AppData\Local\pypoetry\Cache\virtualenvs\mylangchain-app-SBe-Yh6W-py3.12\Lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file c:\Users\user\AppData\Local\pypoetry\Cache\virtualenvs\mylangchain-app-SBe-Yh6W-py3.12\Lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


=== 최종 답변 ===
 아메리카노의 가격은 ₩4,500입니다. 주요 원료는 에스프레소와 뜨거운 물로, 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피입니다. 원두 본연의 맛을 잘 느낄 수 있으며, 깔끔하고 깊은 풍미가 특징입니다. 설탕이나 시럽을 추가할 수 있습니다.

=== 도구 호출 내역 ===
 [{'name': 'db_search_cafe_func', 'args': {'query': '아메리카노'}, 'id': 'call_MLEsEkIcg5qI2sgwCFmTqjU2', 'type': 'tool_call'}, {'name': 'wiki_summary', 'args': {'topic': 'Americano'}, 'id': 'call_qrkJLlxZAlGw6ZsEIQAJ1xts', 'type': 'tool_call'}]

=== 도구 실행 결과(요약) ===
- db_search_cafe_func: [{"page_content": "1. 아메리카노\n   • 가격: ₩4,500\n   • 주요 원료: 에스프레소, 뜨거운 물\n   • 설명: 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피입니다. 원두 본연의 맛을 가장 잘 느낄 수 있으며, 깔끔하고 깊은 풍미가 특징입니다. 설탕이나 시럽 추가 가능합니다....
- wiki_summary: 위키 요약 실패: "아메리카노" may refer to: 
카페 아메리카노
아메리카노
10cm
Americano (노래)
사비에르 쿠가트
아메리카노 (2005년 영화)
아메리카노 (2011년 영화)
아메리카누 FC
제목에 "아메리카노" 항목을 포함한 모든 문서
아메리카나
아메리칸...
